In [ ]:
import numpy as np
import matplotlib.pyplot as plt

**Oppgave 3**

Vi har i filene layers.py og neural_network definert nødvendige klasser med tilhørende medlemsfunksjoner til å trene en transformermodell til å predikere neste tall i en sekvens. Vi skal gjøre dette og begynner med å trene en modell til å kunne addere to heltall. Parameterne skal da oppdateres slik at loss funksjonen blir mindre og mindre. Vi plotter gjennomsnittet av lossfunksjonen for ... som funksjon av antall iterasjoner og ser at dette blir tilfelle.

In [1]:
from training_algorithm import TrainingAlgorithm, prosentSortetRight

nn_adding, mean_losses_addding = TrainingAlgorithm(0)

n = np.arange(1, n+1)

plt.xlabel('Antall iterasjoner')
plt.ylabel('Gjennomsnittlig verdi av L')

plt.plot(n, mean_losses_adding)

'''
prosent = prosentSortetRight(nn_adding)
print(f'Prosentandel av input som ble riktig med nåværende modell: {prosent}')
'''

TypeError: 'float' object is not callable